In [0]:
import os
import mectools.data as dt
import PIL

In [0]:
plt = plotter(backend='Agg')
%matplotlib inline

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
# args
seed = 2384923
samp = 0.01
BATCH_SIZE = 32
size = 1024

In [0]:
def load_path(tag, base='tiles/density', ext='jpg'):
    tag = f'{tag:07d}'
    sub = tag[:4]
    return f'{base}/{size}px/{sub}/{tag}.{ext}'

In [0]:
firms = pd.read_csv('firms/census_2004_geocode.csv', usecols=['id', 'industry', 'income', 'total_assets', 'employees'])
targ = pd.read_csv('targets/census_firms_2004.csv', usecols=['id', 'lat_wgs84', 'lon_wgs84'])
firms = pd.merge(firms, targ, on='id', how='left').dropna()

In [0]:
firms['file'] = firms['id'].apply(load_path)
firms['fexist'] = firms['file'].apply(os.path.exists)
firms = firms[firms['fexist']].drop('fexist', axis=1)

In [0]:
firms['prod'] = firms['income']/firms['employees']
firms['lprod'] = dt.log(firms['prod'])
firms = firms.dropna(subset=['lprod'])

In [0]:
density = np.stack([np.array(PIL.Image.open(fn)) for fn in firms['file']])

In [0]:
firms['dense_128'] = density.reshape((len(firms), -1)).mean(axis=1)

In [0]:
firms['dense_128'].hist();